In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) # your google drive will be mounted as '/content/gdrive/My Drive/*'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls

gdrive	sample_data


In [4]:
import os
os.chdir("/content/gdrive/My Drive/enet")
!ls

args.py   LICENSE  models	README.md	  save	    transforms.py
data	  main.py  mysave	requirements.txt  test.py   utils.py
datasets  metric   __pycache__	run_bash.ipynb	  train.py


In [5]:
# #!/usr/bin/env python
# import subprocess

# !chmod +x ./create_lists.sh

# #call a bash script
# subprocess.call(['./create_lists.sh'])
!ls models

enet_dilation_1.py		enet_no_regularen.py  __pycache__
enet_no_allmaxpool_identity.py	enet_original.py
enet_no_inimaxpool.py		enet.py


### Design Choice Trarining

In [7]:
!python main.py -m train --save-dir mysave/camvid/ --name myenet_no_allmaxpool_y_id_2 --dataset camvid --dataset-dir datasets/camvid/


Loading dataset...

Selected dataset: camvid
Dataset directory: datasets/camvid/
Save directory: mysave/camvid/
Number of classes to predict: 12
Train dataset size: 367
Validation dataset size: 101
Image size: torch.Size([10, 3, 360, 480])
Label size: torch.Size([10, 360, 480])
Class-color encoding: OrderedDict([('sky', (128, 128, 128)), ('building', (128, 0, 0)), ('pole', (192, 192, 128)), ('road', (128, 64, 128)), ('pavement', (60, 40, 222)), ('tree', (128, 128, 0)), ('sign_symbol', (192, 128, 128)), ('fence', (64, 64, 128)), ('car', (64, 0, 128)), ('pedestrian', (64, 64, 0)), ('bicyclist', (0, 128, 192)), ('unlabeled', (0, 0, 0))])

Weighing technique: ENet
Computing class weights...
(this can take a while depending on the dataset size)
Class weights: tensor([ 5.7920,  4.4403, 34.0217,  3.4469, 15.9119,  9.0202, 32.0138, 32.4789,
        13.2071, 38.3877, 44.1345,  0.0000], device='cuda:0')

Training...

ENet(
  (initial_block): InitialBlock(
    (main_branch): Conv2d(3, 16, kernel

### Design Choices Test

In [8]:
!python main.py -m test --save-dir mysave/camvid/ --name myenet_no_allmaxpool_y_id_2 --dataset camvid --dataset-dir datasets/camvid/


Loading dataset...

Selected dataset: camvid
Dataset directory: datasets/camvid/
Save directory: mysave/camvid/
Number of classes to predict: 12
Train dataset size: 367
Validation dataset size: 101
Image size: torch.Size([10, 3, 360, 480])
Label size: torch.Size([10, 360, 480])
Class-color encoding: OrderedDict([('sky', (128, 128, 128)), ('building', (128, 0, 0)), ('pole', (192, 192, 128)), ('road', (128, 64, 128)), ('pavement', (60, 40, 222)), ('tree', (128, 128, 0)), ('sign_symbol', (192, 128, 128)), ('fence', (64, 64, 128)), ('car', (64, 0, 128)), ('pedestrian', (64, 64, 0)), ('bicyclist', (0, 128, 192)), ('unlabeled', (0, 0, 0))])

Weighing technique: ENet
Computing class weights...
(this can take a while depending on the dataset size)
Class weights: tensor([ 5.7920,  4.4403, 34.0217,  3.4469, 15.9119,  9.0202, 32.0138, 32.4789,
        13.2071, 38.3877, 44.1345,  0.0000], device='cuda:0')
ENet(
  (initial_block): InitialBlock(
    (main_branch): Conv2d(3, 16, kernel_size=(3, 3), 

In [0]:
# !python main.py -m train --resume --save-dir mysave/camvid/ --name myenet --dataset camvid --dataset-dir datasets/camvid/

### Input and Output of Each Block

In [0]:
import torch
import torch.optim as optim
from torchsummary import summary
from models.enet import ENet 
import utils

device = torch.device('cuda')
model = ENet(12).to(device)
optimizer = optim.Adam(model.parameters())
model = utils.load_checkpoint(model, optimizer, 'mysave/camvid/',
                                      'myenet')[0]
# print(model)
summary(model.initial_block,(3, 360, 480))
summary(model.downsample1_0,(16, 180, 240))
# summary(model.regular1_1,(64, 90, 120))
summary(model.downsample2_0,(64, 90, 120))
# summary(model.regular3_0,(128, 45, 60))
# summary(model.upsample4_0,(128, 45, 60))
# summary(model.upsample5_0,(64, 90, 120))
#summary(model.transposed_conv,(16, 180, 240)) => (12, 360, 480)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 13, 180, 240]             351
         MaxPool2d-2          [-1, 3, 180, 240]               0
       BatchNorm2d-3         [-1, 16, 180, 240]              32
             PReLU-4         [-1, 16, 180, 240]               1
Total params: 384
Trainable params: 384
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.98
Forward/backward pass size (MB): 15.82
Params size (MB): 0.00
Estimated Total Size (MB): 17.80
----------------------------------------------------------------
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         MaxPool2d-1  [[-1, 16, 90, 120], [-1, 16, 90, 120]]               0
            Conv2d-2           [-1, 4, 90, 120]             256
       BatchNorm2d-3           [-1